In [16]:
from pyperclip import copy

In [39]:
project_id = "qwiklabs-gcp-01-33be2fcab850"

In [40]:
# essential tables
copy(f"""
create schema `{project_id}`.drl;

LOAD DATA OVERWRITE drl.events
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/events.csv']);


create or replace table drl.events as
select
*
from drl.events ;


LOAD DATA OVERWRITE drl.event_pilots
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/event_pilots.csv']);


create or replace table drl.event_pilots as
select
*
from drl.event_pilots ;


LOAD DATA OVERWRITE drl.pilots
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/pilots.csv']);


create or replace table drl.pilots as
select
*
from drl.pilots ;


LOAD DATA OVERWRITE drl.round_standings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/round_standings.csv']);


create or replace table drl.round_standings as
select
*
from drl.round_standings ;

""")

In [41]:
# 2nd essential tables
copy("""
LOAD DATA OVERWRITE drl.time_trial_group_pilot_times
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/time_trial_group_pilot_times.csv']);


create or replace table drl.time_trial_group_pilot_times as
select
*
from drl.time_trial_group_pilot_times ;


LOAD DATA OVERWRITE drl.time_trial_group_pilots
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/time_trial_group_pilots.csv']);


create or replace table drl.time_trial_group_pilots as
select
*
from drl.time_trial_group_pilots ;


LOAD DATA OVERWRITE drl.time_trial_groups
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/time_trial_groups.csv']);


create or replace table drl.time_trial_groups as
select
*
from drl.time_trial_groups ;


LOAD DATA OVERWRITE drl.rounds
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/rounds.csv']);


create or replace table drl.rounds as
select
*
from drl.rounds ;


LOAD DATA OVERWRITE drl.heat_standings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/heat_standings.csv']);


create or replace table drl.heat_standings as
select
*
from drl.heat_standings ;
""")

In [42]:
city_name = "New York"
event_name = "Boston Foundry"

In [43]:
# query batch 1
copy(f"""

select
distinct
name
from `drl.events`
where city = '{city_name}';


select
distinct
b.name as name,
a.id as event_pilot_id
from `drl.event_pilots` a
left join `drl.pilots` b on a.pilot_id = b.id;


select
distinct
b.name as pilot_name,
c.name as event_name
from `drl.event_pilots` a
left join `drl.pilots` b on a.pilot_id = b.id
inner join `drl.events` c on a.event_id = c.id and c.name = '{event_name}';


SELECT time
 (timestamp_seconds
   (CAST
     (AVG
       (UNIX_SECONDS
         (PARSE_TIMESTAMP('%H:%M.%S', minimum_time))
       )
   AS INT64)
   )
 ) as avg
from `drl.round_standings`
where rank = 1;

""")

In [44]:
pilot_name = "NURK"

In [45]:
# query batch 2
copy(f"""

create table drl.time_trial_cleaned as
select
a.id as time_trial_group_pilot_times_id,
a.time_trial_group_pilot_id,
b.time_trial_group_id,
c.round_id,
case 
when a.time_adjusted is not null then a.time_adjusted
when c.racestack_scoring = 0 then a.time
else a.racestack_time end as time
from drl.time_trial_group_pilot_times a
left join drl.time_trial_groups c on a.time_trial_group_pilot_id = c.id
left join drl.time_trial_group_pilots b on c.id = b.time_trial_group_id;

select
min(c.time) as fastest_time
from `drl.events` a
left join `drl.rounds` b on b.event_id = a.id
left join drl.time_trial_cleaned c on b.id = c.round_id
where a.name = '{event_name}'
and b.name = 'Time Trials';


select
distinct
c.name as pilot_name,
a.heat_id,
a.minimum_time,
a.points
from `drl.heat_standings` a
left join `drl.event_pilots` b on a.event_pilot_id = b.id
left join `drl.pilots` c on b.pilot_id = c.id
where a.minimum_time <> ""
and a.minimum_time <> 'DNF'
and c.name = '{pilot_name}';

select
distinct
c.name as pilot_name,
a.heat_id,
a.minimum_time,
a.points,
  time 
    (timestamp_seconds
      (CAST
        (AVG
          (UNIX_SECONDS
            (PARSE_TIMESTAMP('%H:%M.%S', a.minimum_time))
          )
        OVER (ORDER BY a.heat_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
      AS INT64)
      )
    )
  AS running_avg
from `drl.heat_standings` a
left join `drl.event_pilots` b on a.event_pilot_id = b.id
left join `drl.pilots` c on b.pilot_id = c.id
where a.minimum_time <> ""
and a.minimum_time <> 'DNF'
and c.name = '{pilot_name}';

select
pilot_name,
heat_id,
minimum_time,
points,
running_avg,
TIME_DIFF(PARSE_TIME('%H:%M.%S', minimum_time), running_avg, SECOND) as time_diff_from_avg
from (
  select
  distinct
  c.name as pilot_name,
  a.heat_id,
  a.minimum_time,
  a.points,
    time 
      (timestamp_seconds
        (CAST
          (AVG
            (UNIX_SECONDS
              (PARSE_TIMESTAMP('%H:%M.%S', a.minimum_time))
            )
          OVER (ORDER BY a.heat_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
        AS INT64)
        )
      )
    AS running_avg
  from `drl.heat_standings` a
  left join `drl.event_pilots` b on a.event_pilot_id = b.id
  left join `drl.pilots` c on b.pilot_id = c.id
  where a.minimum_time <> ""
  and a.minimum_time <> 'DNF'
  and c.name = '{pilot_name}'
)
""")

In [46]:
# Non essential tables 1
copy("""
LOAD DATA OVERWRITE drl.colors
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/colors.csv']);


create or replace table drl.colors as
select
*
from drl.colors ;


LOAD DATA OVERWRITE drl.event_crossings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/event_crossings.csv']);


create or replace table drl.event_crossings as
select
*
from drl.event_crossings ;


LOAD DATA OVERWRITE drl.event_standings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/event_standings.csv']);


create or replace table drl.event_standings as
select
*
from drl.event_standings ;


LOAD DATA OVERWRITE drl.heat_pilot_crossings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/heat_pilot_crossings.csv']);


create or replace table drl.heat_pilot_crossings as
select
*
from drl.heat_pilot_crossings ;


LOAD DATA OVERWRITE drl.heat_pilots
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/heat_pilots.csv']);


create or replace table drl.heat_pilots as
select
*
from drl.heat_pilots ;

"""
)

In [47]:
# Non essential table 2
copy("""

LOAD DATA OVERWRITE drl.heats
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/heats.csv']);


create or replace table drl.heats as
select
*
from drl.heats ;


LOAD DATA OVERWRITE drl.pilot_asset_playlists
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/pilot_asset_playlists.csv']);


create or replace table drl.pilot_asset_playlists as
select
*
from drl.pilot_asset_playlists ;


LOAD DATA OVERWRITE drl.pilot_assets
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/pilot_assets.csv']);


create or replace table drl.pilot_assets as
select
*
from drl.pilot_assets ;


LOAD DATA OVERWRITE drl.pilot_season_stats
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/pilot_season_stats.csv']);


create or replace table drl.pilot_season_stats as
select
*
from drl.pilot_season_stats ;


LOAD DATA OVERWRITE drl.race_standings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/race_standings.csv']);


create or replace table drl.race_standings as
select
*
from drl.race_standings ;


LOAD DATA OVERWRITE drl.races
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/races.csv']);


create or replace table drl.races as
select
*
from drl.races ;


LOAD DATA OVERWRITE drl.scorings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/scorings.csv']);


create or replace table drl.scorings as
select
*
from drl.scorings ;


LOAD DATA OVERWRITE drl.seasons
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/seasons.csv']);


create or replace table drl.seasons as
select
*
from drl.seasons ;


LOAD DATA OVERWRITE drl.time_trial_group_pilot_crossings
FROM FILES (
format = 'CSV',
uris = ['gs://spls/gsp394/tables/time_trial_group_pilot_crossings.csv']);


create or replace table drl.time_trial_group_pilot_crossings as
select
*
from drl.time_trial_group_pilot_crossings ;

""")

In [48]:
# if and only if data studio one fails
copy("""

select 
c.name as name, 
a.minimum_time as minimum_time, 
a.heat_id as heat_id, 
safe_cast(a.points as integer) as points 
from drl.heat_standings a 
join drl.event_pilots b on a.event_pilot_id = b.id 
join drl.pilots c on b.pilot_id = c.id 
 
where minimum_time not in ( "DNF" , "0") 
and points != 0
order by heat_id asc;
"""
)